In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
#import keras
from neuroner import neuromodel
from shutil import copyfile, copy
import random
import os
import pandas as pd

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

### Functions charged with formatting the data set in the original format

In [7]:
def to_brat_eHealth_format(path, file):
    temp = pd.read_csv(path+file, sep='\t', index_col=0, names=['label', 'txt'])
    _temp = temp.label.str.split(expand=True, n=1)
    temp['label'] = _temp[0]
    temp['spans'] = _temp[1]
    if temp.index.str.contains('T', regex=False).any(): temp.index = [i[1:] for i in temp.index]
        
    offset_to_change = temp.loc[temp.txt.str.split().str.len() > 1]
    offset_to_change.txt = offset_to_change.txt.str.split()
    offset_to_change.spans = offset_to_change.spans.str.split()
    offset_to_change['relative_spans'] = [[len(elem) for elem in row] for row in offset_to_change.txt]
    
    for i in range(len(offset_to_change)):
        current = int(offset_to_change.spans[i][0])
        end = int(offset_to_change.spans[i][-1])
        relative_spans = offset_to_change.relative_spans[i]
        new_spans = []
        for span in relative_spans:
            new_spans.extend((str(current), str(current+span)+';'))
            current = current+span+1
        if int(new_spans[-1][:-1])!=end: print ("diferentes tokens sin separar por espacio en: ", offset_to_change.index[i], new_spans[-1][:-1], end)
        #print (' '.join(new_spans)[:-1].replace('; ', ';'))
        temp.spans[temp.index == offset_to_change.index[i]] = ' '.join(new_spans)[:-1].replace('; ', ';')
    print (temp[['spans', 'label', 'txt']])
    temp[['spans', 'label', 'txt']].to_csv(path+file, sep='\t', header=False)

def paths_brat_eHealth_format(paths):    
    for path in paths:
        for f in os.listdir(path):
            if os.path.isfile(os.path.join(path, f)) and f.endswith('.ann'):
                to_brat_eHealth_format(path, f)

The path of the model to be used to predict must be indicated below.

In [3]:
model = "./output/(path on which the model to be evaluated was stored)"

## Loading pretrained model 
All parameters are taken from the file "parameters.ini".

In [4]:
nn = neuromodel.NeuroNER(train_model=False, use_pretrained_model=True, 
                         pretrained_model_folder=model)

{'train_model': 0, 'use_pretrained_model': 1, 'pretrained_model_folder': './models/run2/', 'output_scores': 0, 'dataset_text_folder': './data/', 'main_evaluation_mode': 'token', 'output_folder': './output', 'use_character_lstm': 1, 'character_embedding_dimension': 25, 'character_lstm_hidden_state_dimension': 25, 'token_pretrained_embedding_filepath': '', 'token_embedding_dimension': 200, 'token_lstm_hidden_state_dimension': 200, 'use_crf': 1, 'patience': 100, 'maximum_number_of_epochs': 100, 'optimizer': 'sgd', 'learning_rate': 0.001, 'gradient_clipping_value': 0.0, 'dropout_rate': 0.5, 'number_of_cpu_threads': 8, 'number_of_gpus': 1, 'experiment_name': 'test', 'tagging_format': 'bioes', 'tokenizer': 'spacy', 'spacylanguage': 'es', 'remap_unknown_tokens_to_unk': 1, 'load_only_pretrained_token_embeddings': 0, 'check_for_lowercase': 1, 'check_for_digits_replaced_with_zeros': 1, 'freeze_token_embeddings': 0, 'debug': 0, 'verbose': 0, 'plot_format': 'png', 'reload_character_embeddings': 1,

In [5]:
nn.fit()


Starting epoch 0
Training completed in 0.00 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     0.8505    0.8618    0.8561       977
     Concept     0.9438    0.9262    0.9349      2899
   Predicate     0.7529    0.5773    0.6535       343
   Reference     0.9048    0.5802    0.7070       131

   micro avg     0.9089    0.8738    0.8910      4350
   macro avg     0.8630    0.7364    0.7879      4350
weighted avg     0.9066    0.8738    0.8881      4350

Evaluate model on the valid set
              precision    recall  f1-score   support

      Action     0.7531    0.7262    0.7394       168
     Concept     0.8660    0.9085    0.8867       448
   Predicate     0.6667    0.4783    0.5570        46
   Reference     0.9091    0.4167    0.5714        24

   micro avg     0.8299    0.8178    0.8238       686
   macro avg     0.7987    0.6324    0.6886       686
weighted avg     0.8265    0.8178    0.8175       686

Predict label

## Postprocess task

In [ ]:
path_to_process = "./output/"+model+"/000_deploy.txt" # the path of the output file on which the entities are predicted by NeuroNER model
original_brat = "./data/deploy/" # the path where we locate the test dataset
brat_output = "./data/deploy/" # the path on which the processed file is going to be saved on brat format

neuromodel.conll_to_brat.conll_to_brat(path_to_process, path_to_process, original_brat, brat_output, overwrite=True)
paths_brat_eHealth_format([brat_output])

Once the execution has finished, the final output should be placed in the folder indicated by "brat_output"